In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.nn.functional as F

In [2]:
import joblib

X_uncat = joblib.load('X_uncat.pkl')
y = joblib.load('y.pkl')

In [3]:
X_uncat.shape

(65034, 675, 14)

In [4]:
y.shape

(65034,)

In [7]:
import pandas as pd

df = pd.read_csv('./mindbig_final_df.csv')

In [9]:
df.head()

,id,event,device,channel,code,size,alpha_0,alpha_1,alpha_2,alpha_3,...,sigma_125,sigma_126,sigma_127,sigma_128,sigma_129,sigma_130,sigma_131,sigma_132,sigma_133,sigma_134
0,67635,67635,EP,AF3,6,260,0.319863,2.441277,8.833889,20.215495,...,-12.958829,-25.521999,-37.618403,-48.813262,-58.610222,-66.487943,-71.946180,-74.557220,-74.016620,-70.186705
1,67636,67635,EP,F7,6,260,0.326693,2.493346,9.022391,20.648117,...,-13.054094,-26.035788,-38.506702,-50.026377,-60.097178,-68.198218,-73.828418,-76.556006,-76.069798,-72.224948
2,67637,67635,EP,F3,6,260,0.330275,2.520999,9.123475,20.880613,...,-14.996782,-28.121353,-40.591954,-51.976920,-61.793656,-69.542278,-74.749714,-77.019742,-76.082220,-71.833860
3,67638,67635,EP,FC5,6,260,0.306204,2.337773,8.462327,19.371563,...,-13.416258,-25.957415,-37.886198,-48.778552,-58.168924,-65.583350,-70.579782,-72.791892,-71.971329,-68.023447
4,67639,67635,EP,T7,6,260,0.327327,2.499235,9.047719,20.715055,...,-13.066634,-25.760951,-37.926896,-49.151228,-58.963620,-66.871657,-72.403903,-75.156337,-74.837783,-71.309596


In [11]:
last_column = df.columns[-1]
print(last_column)
final_index = last_column.split('_')[-1]
print(final_index)

sigma_134
134


In [ ]:
# our X should be size (N, 135, 14 * 4)
alpha_columns = [f'alpha_{i}' for i in range(0, int(final_index) + 1)]
beta_columns = [f'beta_{i}' for i in range(0, int(final_index) + 1)]
theta_columns = [f'theta_{i}' for i in range(0, int(final_index) + 1)]
delta_columns = [f'delta_{i}' for i in range(0, int(final_index) + 1)]
sigma_columns = [f'sigma_{i}' for i in range(0, int(final_index) + 1)]

# columns that are not the above
other_columns = [col for col in df.columns if col not in alpha_columns and col not in beta_columns and col not in theta_columns and col not in delta_columns and col not in sigma_columns]

print(other_columns)



['id', 'event', 'device', 'channel', 'code', 'size']


In [20]:
# Define the base DataFrame and frequency bands
base_df = df[other_columns]
bands = {
    "alpha": alpha_columns,
    "beta": beta_columns,
    "theta": theta_columns,
    "delta": delta_columns,
    "sigma": sigma_columns
}

# Create DataFrames for each frequency band
band_dfs = {band: base_df.join(df[columns]) for band, columns in bands.items()}

# Print shapes of the band DataFrames
print(*(f"{band}: {band_dfs[band].shape}" for band in bands))

# Extract unique channels
channels = base_df["channel"].unique()
print(channels)

# Extract data for each band and channel into a dictionary
band_channel_data = {
    band: {
        channel: band_dfs[band][band_dfs[band]["channel"] == channel][columns].values
        for channel in channels
    }
    for band, columns in bands.items()
}

seq_length = 135

# Create a 3D tensor with shape (N, seq_length, 14 * 5)
X = np.concatenate(
    [
        np.stack([band_channel_data[band][channel] for channel in channels], axis=-1)
        for band in bands
    ],
    axis=-1
)

# Reshape to ensure the sequence length is seq_length
X = X[:, :seq_length, :]  # Truncate or pad if necessary to match seq_length

print(f"Shape of X: {X.shape}")


alpha: (910476, 141) beta: (910476, 141) theta: (910476, 141) delta: (910476, 141) sigma: (910476, 141)
['AF3' 'F7' 'F3' 'FC5' 'T7' 'P7' 'O1' 'O2' 'P8' 'T8' 'FC6' 'F4' 'F8' 'AF4']
Shape of X: (65034, 135, 70)


In [21]:
print(X.shape, y.shape)

(65034, 135, 70) (65034,)


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.nn.functional as F

# apple gpu 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
    device = mps_device
else:
    print ("MPS device not found.")


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train = X_train.transpose(0, 2, 1)
# X_test = X_test.transpose(0, 2, 1)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


# Define the PyTorch model
class EEGNet(nn.Module):
    def __init__(self, channels=14, n_classes=11):
        super(EEGNet, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(channels)  # Channels should match input shape
        self.conv1 = nn.Conv1d(channels, 128, kernel_size=10, stride=1, padding='same')
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(2)
        self.lstm = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.batch_norm3 = nn.BatchNorm1d(256)
        self.fc1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(128, n_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Switch to (batch, channels, sequence)
        x = self.batch_norm1(x)
        x = F.relu(self.conv1(x))
        x = self.batch_norm2(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)  # Reorder for LSTM (batch, sequence, input_size)
        x, _ = self.lstm(x)
        x = self.batch_norm3(x[:, -1, :])  # Use the last LSTM output
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.output(x)
        return F.log_softmax(x, dim=1)


tensor([1.], device='mps:0')
(52027, 135, 70) (52027,) (13007, 135, 70) (13007,)


In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model = EEGNet(channels=70, n_classes=11).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping criteria
best_accuracy = 0
patience = 10
trigger_times = 0

print(device)

# Training loop
for epoch in range(10):
    model.train()
    for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X.to(device))
        loss = criterion(outputs, batch_Y.to(device))
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_train_tensor.to(device))
        val_loss = criterion(val_outputs, y_train_tensor.to(device))
        _, val_preds = torch.max(val_outputs, 1)
        val_accuracy = accuracy_score(y_train_tensor.cpu().numpy(), val_preds.cpu().numpy())
        print(f"Epoch {epoch+1}, Validation Accuracy: {val_accuracy}")

        # Early stopping logic
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            trigger_times = 0
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping")
                break


mps


RuntimeError: Placeholder storage has not been allocated on MPS device!